In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity

adarDamageSeqDataDirectory = os.path.join(getDataDir(), "Adar_damage-seq")

dataSets = ["A549_BPDE_cell_2h_A"]

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

adarDamageSeqRawCounts = dict()
adarDamageSeqNormalizedCounts = dict()

TFBS_Names = ["hg19_CTCF_known", "hg19_ETS_and_GABPA_known", "hg19_SP1_known"]

for dataSet in dataSets:

    adarDamageSeqRawCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
    adarDamageSeqNormalizedCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}

    for nucleosomeCountsFilePath in getFilesInDirectory(os.path.join(adarDamageSeqDataDirectory, dataSet), DataTypeStr.rawNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in TFBS_Names: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        adarDamageSeqRawCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    for nucleosomeCountsFilePath in getFilesInDirectory(os.path.join(adarDamageSeqDataDirectory, dataSet), DataTypeStr.normNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in TFBS_Names: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        adarDamageSeqNormalizedCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

In [ ]:
for dataSet in dataSets:
    for radius in adarDamageSeqRawCounts[dataSet]:
        for TFBS_Name in adarDamageSeqRawCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(adarDamageSeqRawCounts[dataSet][radius][TFBS_Name], dataCol = "Both_Strands_Counts",
                                           smoothTranslational = False, nucRepLen = 2000,
                                           title = f"Adar {dataSet} {TFBS_Name} Raw Counts", yAxisLabel = "Damage Reads")
            print(plot)

In [ ]:
for dataSet in dataSets:
    for radius in adarDamageSeqNormalizedCounts[dataSet]:
        for TFBS_Name in adarDamageSeqNormalizedCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(adarDamageSeqNormalizedCounts[dataSet][radius][TFBS_Name], dataCol = "Normalized_Both_Strands",
                                           smoothTranslational = False, nucRepLen = 2000,
                                           title = f"Adar {dataSet} {TFBS_Name} Naked Normalized", yAxisLabel = "Cellular/Naked Damage Reads")
            print(plot)